In [30]:
import pandas as pd
import numpy as np

In [31]:
headlines = pd.read_csv('./data/raw/abcnews-date-text.csv')
mm = pd.read_csv('./data/raw/dataverse_files/mmALL_073120_csv.csv')
wgi = pd.read_csv('./data/raw/Data_Extract_From_Worldwide_Governance_Indicators/3f2d17f2-cc70-48f7-8fb9-ab3d88c6b0bd_Data.csv')


In [32]:
headlines['publish_date']=pd.to_datetime(headlines['publish_date'], format="%Y%m%d")

In [33]:
headlines.set_index('publish_date', drop=True, inplace=True)

In [34]:
headlines.head()

,headline_text
publish_date,
2003-02-19,aba decides against community broadcasting lic...
2003-02-19,act fire witnesses must be aware of defamation
2003-02-19,a g calls for infrastructure protection summit
2003-02-19,air nz staff in aust strike for pay rise
2003-02-19,air nz strike to affect australian travellers


#### Comparing country names in wgi and mm. 

The following countries are not represented the same in wgi, and I will check for spelling discrepancies:
* 'Venezuela', 'Macedonia', 'Bosnia', 'Yugoslavia',
       'Serbia and Montenegro', 'Russia', 'Cape Verde', 'Gambia',
       'Ivory Coast', 'Congo Brazzaville', 'Congo Kinshasa', 'Swaziland',
       'Iran', 'Egypt', 'Syria', 'Yemen', 'United Arab Emirate',
       'Kyrgyzstan', 'Taiwan', 'North Korea', 'South Korea',
       'Timor Leste'
       * rename wgi 'Venezuela, RB' to 'Venezuela'
       * rename mm Macedonia to North Macedonia
       * rename mm Bosnia to 'Bosnia and Herzegovina'
       * rename mm Yugoslavia to Serbia
       * rename mm Serbia and Montenegro to ??? Serbia? Montenegro? 
       * rename mm Russia to Russian Federation
       * rename mm cape verde to Cabo Verde
       * rename mm Gambia to 'Gambia, The'
       * rename mm Ivory Coast to  'Cote d'Ivoire'
       * rename mm Congo Brazzaville to 'Congo, Rep'
       * rename mm Congo Kinshasa to 'Congo, Dem. Rep.'
       * rename mm Swaziland to 'Eswatini'
       * rename wgi 'Iran, Islamic Rep.' to 'Iran'
       * rename wgi 'Egypt, Arab Rep.' to 'Egypt'
       * rename wgi Syrian Arab Republic to 'Syria'
       * rename wgi 'Yemen, Rep.' to 'Yemen'
       * rename mm 'United Arab Emirate' to 'United Arab Emirates'
       * rename wgi 'Kyrgyz Republic' to 'Kyrgyzstan'
       * rename wgi 'Taiwan, China' to 'Taiwan'
       * rename wgi "Korea, Dem. People's Rep." to 'North Korea'
       * rename wgi 'Korea, Rep.' to 'South Korea'
       * rename mm 'Timor Leste' to 'Timor-Leste'
       
       
       
       
       


In [35]:
mm.head()

,id,country,ccode,year,region,protest,protestnumber,startday,startmonth,startyear,...,protesterdemand4,stateresponse1,stateresponse2,stateresponse3,stateresponse4,stateresponse5,stateresponse6,stateresponse7,sources,notes
0,201990001,Canada,20,1990,North America,1,1,15.0,1.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. great canadian train journeys into history;...,canada s railway passenger system was finally ...
1,201990002,Canada,20,1990,North America,1,2,25.0,6.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. autonomy s cry revived in quebec the new yo...,protestors were only identified as young peopl...
2,201990003,Canada,20,1990,North America,1,3,1.0,7.0,1990.0,...,NaN,ignore,NaN,NaN,NaN,NaN,NaN,NaN,1. quebec protest after queen calls for unity ...,"the queen, after calling on canadians to remai..."
3,201990004,Canada,20,1990,North America,1,4,12.0,7.0,1990.0,...,NaN,accomodation,NaN,NaN,NaN,NaN,NaN,NaN,1. indians gather as siege intensifies; armed ...,canada s federal government has agreed to acqu...
4,201990005,Canada,20,1990,North America,1,5,14.0,8.0,1990.0,...,NaN,crowd dispersal,arrests,accomodation,NaN,NaN,NaN,NaN,1. dozens hurt in mohawk blockade protest the ...,protests were directed against the state due t...


In [36]:
wgi.head()

,Country Name,Country Code,Series Name,Series Code,1996 [YR1996],1998 [YR1998],2000 [YR2000],2002 [YR2002],2003 [YR2003],2004 [YR2004],...,2011 [YR2011],2012 [YR2012],2013 [YR2013],2014 [YR2014],2015 [YR2015],2016 [YR2016],2017 [YR2017],2018 [YR2018],2019 [YR2019],2020 [YR2020]
0,Afghanistan,AFG,Control of Corruption: Estimate,CC.EST,-1.291705,-1.180848,-1.29538,-1.263366,-1.351042,-1.345281,...,-1.579174,-1.419741,-1.43651,-1.354829,-1.342216,-1.526172,-1.515626,-1.487624,-1.400733,-1.475405
1,Afghanistan,AFG,Control of Corruption: Number of Sources,CC.NO.SRC,2,2,2,2,3,5,...,9,10,11,11,11,10,10,10,10,9
2,Afghanistan,AFG,Control of Corruption: Percentile Rank,CC.PER.RNK,4.301075,9.793815,5.076142,5.050505,5.050505,5.853659,...,0.9478673,2.369668,1.895735,5.288462,6.25,3.365385,3.846154,4.807693,6.730769,5.288462
3,Afghanistan,AFG,"Control of Corruption: Percentile Rank, Lower ...",CC.PER.RNK.LOWER,0,0,0,0,0,0,...,0,0,0,0.4807692,1.442308,0,0,0,1.923077,0
4,Afghanistan,AFG,"Control of Corruption: Percentile Rank, Upper ...",CC.PER.RNK.UPPER,27.41936,31.4433,29.44162,31.81818,18.18182,14.14634,...,5.687204,11.84834,9.952606,12.5,12.5,9.615385,9.615385,10.09615,11.53846,11.05769


In [37]:
# comparing country names in wgi and mm. 

missing_countries_not_in_mm = list(mm[~mm['country'].isin(wgi['Country Name'])]['country'].unique())
missing_countries_not_in_mm[:5]

['Venezuela', 'Germany West', 'Germany East', 'Czechoslovakia', 'Macedonia']

In [38]:
missing_countries_not_in_wgi = list(wgi[~wgi['Country Name'].isin(mm['country'])]['Country Name'].unique())
missing_countries_not_in_wgi[:5]

['American Samoa', 'Andorra', 'Anguilla', 'Antigua and Barbuda', 'Aruba']

In [39]:
def clean_mass_mob_df(df=mm):
    
    #Drop nulls in startday/month/year and endday/month/year:
    df = df.drop(df[df["startyear"].isnull()].index)
    df.reset_index(inplace=True)
    
    #set startyear/month/day and endday/month/year to int, then string, in preparation for concatenation
    df = df.astype({"startyear": int, "startmonth": int, 'startday': int, "endyear": int, "endmonth": int, 'endday': int})
    df = df.astype({"startyear": str, "startmonth": str, 'startday': str, "endyear": str, "endmonth": str, 'endday': str})
    
    #create start date and end date columns and set as datetime object
    df['start_date'] = df['startyear'] + '-' + df['startmonth'] + '-'+ df['startday']
    df['end_date'] = df['endyear'] + '-' + df['endmonth'] + '-'+ df['endday']
    df['start_date'] = pd.to_datetime(df['start_date'], yearfirst=True)
    df['end_date'] = pd.to_datetime(df['end_date'], yearfirst=True)
    
    #setting df to only include dates between 2002 and 2020.
    df = df[(df['start_date']>'2001') & (df['start_date']<'2021')]
    
    #engineer a column to give duration of protest
    df['duration'] = df['end_date'] - df['start_date']
    
    #fill nulls with 'unknown'
    df.fillna('unknown', inplace=True)
    
    #resetting index
    df.reset_index(drop=True,inplace=True)
    
    #replacing country names in mm to align with wgi country names

    country_changes_mm = {'Macedonia': 'North Macedonia',
                          'Bosnia':'Bosnia and Herzegovina',
                          'Yugoslavia':'Serbia',
                          'Serbia and Montenegro': 'Montenegro',
                          'Russia': 'Russian Federation',
                          'Cape Verde': 'Cabo Verde',
                          'Gambia': 'Gambia, The',
                          'Ivory Coast': 'Cote d\'Ivoire',
                          'Congo Brazzaville': 'Congo, Rep.',
                          'Congo Kinshasa':'Congo, Dem. Rep.',
                          'Swaziland': 'Eswatini',
                          'United Arab Emirate': 'United Arab Emirates',
                          'Timor Leste':'Timor-Leste'}

    for name in mm['country']:
        for k, v in country_changes_mm.items():
            if name == k:
                mm.replace({'country': country_changes_mm}, inplace=True)
        
    #drop original date columns
    df.drop(columns = ['startday', 'startmonth', 'startyear', 'endday', 'endmonth', 'endyear', 'index'], inplace=True)
    
    return df
    

In [40]:
mm = clean_mass_mob_df(mm)

In [41]:
# checking for 60 countries with most appearances in mass mobilization dataset

mm_country_vals = pd.DataFrame(mm['country'].value_counts()[:60])['country'].index
mm_country_vals

Index(['United Kingdom', 'Ireland', 'Kenya', 'Germany', 'China', 'France',
       'Greece', 'Bangladesh', 'Thailand', 'Venezuela', 'South Korea', 'India',
       'Namibia', 'Russia', 'Brazil', 'Kyrgyzstan', 'Nigeria', 'Yemen',
       'Pakistan', 'Italy', 'Ukraine', 'Spain', 'Iraq', 'Nepal', 'Cyprus',
       'Chile', 'Egypt', 'Lebanon', 'Malaysia', 'Turkey', 'Peru', 'Guinea',
       'Iran', 'Cuba', 'Guatemala', 'Algeria', 'Ecuador', 'Romania',
       'Madagascar', 'Mexico', 'Taiwan', 'Haiti', 'Azerbaijan', 'Colombia',
       'South Africa', 'Armenia', 'Honduras', 'Indonesia', 'Moldova',
       'Swaziland', 'Argentina', 'Burundi', 'Bolivia', 'Togo', 'Macedonia',
       'Mauritania', 'Myanmar', 'Liberia', 'Zimbabwe', 'Sudan'],
      dtype='object')

In [42]:
mm['year'].describe()

count    10573.000000
mean      2011.308143
std          5.458907
min       2001.000000
25%       2007.000000
50%       2012.000000
75%       2016.000000
max       2020.000000
Name: year, dtype: float64

For my initial modeling, I will create 6 models based on their freedom rating in 2006:
1. A country that was classified as 'Free' in 2006 and appeared frequently the mass mobilization dataset.
2. A country that was classified as 'Free' in 2006 and appeared sparsely in the mass mobilization dataset.
3. A country that was classified as 'Not Free' in 2006 and appears frequently in the mass mobilization dataset.
4. A country that was classified as 'Not Free' in 2006 and appears sparsely the mass mobilization dataset.
5. A country that was classified as 'Partially Free' in 2006 and appears frequently the mass mobilization dataset.
6. A country that was classified as 'Partially Free' in 2006 and appears sparsely the mass mobilization dataset.

In order to determine which countries to model, I will cross examine the mass mobilization dataset and the Freedom in the World dataset. 


In [43]:
fiw = pd.read_csv('./data/transformed/fiw_agg.csv')

In [44]:
fiw.head()

,Country/Territory,Region,C/T?,Edition,Status,PR Rating,CL Rating,A,B,C,...,Unnamed: 26,Unnamed: 27,Unnamed: 28,Unnamed: 29,Unnamed: 30,Unnamed: 31,Unnamed: 32,Unnamed: 33,Unnamed: 34,x
0,Abkhazia,Eurasia,t,2021,PF,5,5,5,8,4,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Afghanistan,Asia,c,2021,NF,5,6,3,7,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Albania,Europe,c,2021,PF,3,3,8,12,7,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Algeria,MENA,c,2021,NF,6,5,3,4,3,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Andorra,Europe,c,2021,F,1,1,12,15,11,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
fiw.drop(columns=['Country/Territory', 'Status']).columns

Index(['Region', 'C/T?', 'Edition', 'PR Rating', 'CL Rating', 'A', 'B', 'C',
       'Add Q', 'Add A', 'PR', 'D', 'E', 'F', 'G', 'CL', 'Total',
       'Unnamed: 19', 'Unnamed: 20', 'Unnamed: 21', 'Unnamed: 22',
       'Unnamed: 23', 'Unnamed: 24', 'Unnamed: 25', 'Unnamed: 26',
       'Unnamed: 27', 'Unnamed: 28', 'Unnamed: 29', 'Unnamed: 30',
       'Unnamed: 31', 'Unnamed: 32', 'Unnamed: 33', 'Unnamed: 34', 'x'],
      dtype='object')

In [46]:
def clean_fiw_for_model(df=fiw):
    
    columns_to_drop = df.drop(columns=['Country/Territory', 'Status', 'Edition']).columns
    
    df.drop(columns=columns_to_drop, inplace=True)
    
    fiw_column_dict = {}
    for column in df.columns:
        if column == 'Country/Territory':
            fiw_column_dict[column] = 'country'
        elif column == 'C/T?':
            fiw_column_dict[column] = 'country_territory'
        else:
            fiw_column_dict[column] = 'fiw_' + column.lower().replace(' ', '_')

    df.rename(mapper=fiw_column_dict, axis=1, inplace=True)

    country_map = {
        'Slovakia': 'Slovak Republic',
        'Serbia and Montenegro': 'Serbia'   
    }

    for k, v in country_map.items():
        for country in df['country']:
            if country == k:
                country = v
  
    
    df['fiw_edition'] = pd.to_datetime(df['fiw_edition'], format='%Y').dt.year
    
    return df

In [47]:
fiw_clean = clean_fiw_for_model(fiw)

In [48]:
fiw_clean['fiw_edition'] = pd.to_datetime(fiw_clean['fiw_edition'], format='%Y').dt.year

In [50]:
fiw_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3339 entries, 0 to 3338
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   country      3339 non-null   object
 1   fiw_edition  3339 non-null   int64 
 2   fiw_status   3339 non-null   object
dtypes: int64(1), object(2)
memory usage: 78.4+ KB


In [23]:

fiw_clean.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3339 entries, 0 to 3338
Data columns (total 3 columns):
 #   Column       Non-Null Count  Dtype 
---  ------       --------------  ----- 
 0   country      3339 non-null   object
 1   fiw_edition  3339 non-null   int64 
 2   fiw_status   3339 non-null   object
dtypes: int64(1), object(2)
memory usage: 78.4+ KB


In [ ]:
fiw_clean.info()

In [ ]:
fiw_clean.describe()

In [ ]:
#checking for countries categorized as 'Not Free' in 2006

fiw_2006_not_free_list = list(fiw_clean[(fiw_clean['fiw_edition'] == 2006) & (fiw_clean['fiw_status'] == 'NF')]['country'])
fiw_2006_partially_free_list = list(fiw_clean[(fiw_clean['fiw_edition'] == 2006) & (fiw_clean['fiw_status'] == 'PF')]['country'])
fiw_2006_free_list = list(fiw_clean[(fiw_clean['fiw_edition'] == 2006) & (fiw_clean['fiw_status'] == 'F')]['country'])
fiw_2021_not_free_list = list(fiw_clean[(fiw_clean['fiw_edition'] == 2021) & (fiw_clean['fiw_status'] == 'NF')]['country'])
fiw_2021_partially_free_list = list(fiw_clean[(fiw_clean['fiw_edition'] == 2021) & (fiw_clean['fiw_status'] == 'PF')]['country'])
fiw_2021_free_list = list(fiw_clean[(fiw_clean['fiw_edition'] == 2021) & (fiw_clean['fiw_status'] == 'F')]['country'])


In [ ]:
#check for which countries in the various Freedom Scales categories by year occur most frequently in the Mass Mobilization data. 
nf_2006_mm_frequent = []
for country in fiw_2006_not_free_list:
    if country in mm_country_vals:
        nf_2006_mm_frequent.append(country)

print(nf_2006_mm_frequent)


Algeria, China, Cuba, Egypt, Guinea, Iran, Iraq, Nepal, Pakistan, Russia intersect as appearing frequently in the mass mobilization dataset and were given the category of 'Not Free' in 2006. I will begin by reviewing these options for my first model. 

In [ ]:
def get_freedom_status_2006_2021(country_list):
    for country in country_list:
        if country in fiw_2021_free_list:
            print(f'{country} status is "Free" in 2021')
        if country in fiw_2021_partially_free_list:
            print(f'{country} status is "Partially Free" in 2021')
        if country in fiw_2021_not_free_list:
            print(f'{country} status is "Not Free" in 2021')


In [ ]:
get_freedom_status_2006_2021(nf_2006_mm_frequent)

Unfortunately, it doesn't appear that the needle has move significantly for many of these countries, thought we do see a shift to 'partially free' for Guinea and Pakistan.

Based on this, I will begin by modeling Pakistan. 

In [ ]:
pf_2006_mm_frequent = []
for country in fiw_2006_partially_free_list:
    if country in mm_country_vals:
        pf_2006_mm_frequent.append(country)

print(pf_2006_mm_frequent)

In [ ]:
get_freedom_status_2006_2021(pf_2006_mm_frequent)

In [ ]:
f_2006_mm_frequent = []
for country in fiw_2006_free_list:
    if country in mm_country_vals:
        f_2006_mm_frequent.append(country)

print(f_2006_mm_frequent)

In [ ]:
get_freedom_status_2006_2021(f_2006_mm_frequent)

In [ ]:
# Looking for which countries have seen variation in freedom status from 2006-2021

# I referenced stackoverflow for guidance on re-typing dictionary values

status_dict = {}
for i in fiw['country'].unique():
   status_dict[i] = [
         fiw['fiw_status'][j]
      for j in fiw[fiw['country'] == i].index
   ]

for keys in status_dict:
    status_dict[keys] = set(status_dict[keys])
    

In [ ]:
for k, v in status_dict.items():
    if k in nf_2006_mm_frequent:
        print(k, v)

In [ ]:
for k, v in status_dict.items():
    if k in pf_2006_mm_frequent:
        print(k, v)

In [ ]:
for k, v in status_dict.items():
    if k in f_2006_mm_frequent:
        print(k, v)

In [ ]:
for k, v in status_dict.items():
    if len(v) == 3:
        print(k, v)

In [ ]:
mm[mm['country'] == 'Mali'].shape

In [ ]:
mm[mm['country'] == 'Tunisia'].shape

In [ ]:
mm[mm['country'] == 'Venezuela'].shape

In [ ]:
mm[mm['country'] == 'Kyrgyzstan'].shape

In [ ]:
mm[mm['country'] == 'Kazakhstan'].shape

World Governance Indicators I want to use:
* Control of Corruption: Estimate (CC.EST)
    * Control of Corruption captures perceptions of the extent to which public power is exercised for private gain
* Government Effectiveness: Estimate (GE.EST)
    * Government Effectiveness captures perceptions of the quality of public service
* Political Stability and Absence of Violence/Terrorism: Estimate (PV.EST)
    * Political Stability and Absence of Violence/Terrorism measures perceptions of the likelihood of political instability and/or politically-motivated violence
* Rule of Law: Estimate
    * Rule of Law captures perceptions of the extent to which agents have confidence in and abide by the rules of society
* Voice and Accountability: Estimate
    * Voice and Accountability captures perceptions of the extent to which a country's citizens are able to participate in selecting their government

In [ ]:
wgi.head()

In [ ]:
def clean_wgi(df=wgi):
    
    #dropping country code column
    df.drop(columns=['Country Code', 'Series Name'], inplace=True)
    
    #rename year columns
    for column in df.columns[2:]:
        df.rename(columns = {column: column[:4]}, inplace=True)
        
    for column in df.columns[:2]:
        df.rename(columns = {column: column.lower().replace(' ', '_')}, inplace=True)
        
    #create columns for 1997, 1999, and 2001, which will replicate the previous year
    df['1997'] = df['1996']
    df['1999'] = df['1998']
    df['2001'] = df['2000']
    
    #reorder columns
    df = df[['country_name', 'series_code', '1996', '1997', '1998', '1999', '2000',
       '2001','2002', '2003', '2004', '2005', '2006', '2007', '2008', '2009', '2010',
       '2011', '2012', '2013', '2014', '2015', '2016', '2017', '2018', '2019',
       '2020']]
    
    #dropping unneeded series codes
    wgi_series = ['CC.EST', 'GE.EST', 'PV.NO.SRC', 'RL.EST', 'VA.EST']
    df = df[df['series_code'].isin(wgi_series)]
    
    #replacing country names in wgi to align with mm
    country_changes_wgi = {'Venezuela, RB': 'Venezuela',
                          'Iran, Islamic Rep.': 'Iran',
                          'Egypt, Arab Rep.': 'Egypt',
                          'Syrian Arab Republic': 'Syria',
                          'Yemen, Rep.': 'Yemen',
                          'Kyrgyz Republic': 'Kyrgyzstan',
                          'Taiwan, China': 'Taiwan',
                          'Korea, Dem. People\'s Rep.': 'North Korea',
                          'Korea, Rep.': 'South Korea'}

    for name in wgi['country_name']:
        for k, v in country_changes_wgi.items():
            if name == k:
                wgi.replace({'country_name': country_changes_wgi}, inplace=True)
    
    df.reset_index(drop=True, inplace=True)
    
    # using melt to get rows and country_name set for merging with other dataframes
    # referenced stackoverflow for melt documentation: https://stackoverflow.com/questions/28654047/convert-columns-into-rows-with-pandas
    df = df.melt(id_vars=['country_name', 'series_code'])
    df.rename(columns = {'variable': 'year', 'value': 'score'}, inplace=True)
    
    # setting year as datetime object
    df['year'] = pd.to_datetime(df['year'], format='%Y').dt.year
    
    return df

In [ ]:
wgi = clean_wgi()

In [ ]:
wgi.head()

In [ ]:
# for code in wgi['series_code'].unique():
#     wgi[code] = wgi['score']

In [ ]:
wgi

In [ ]:
wgi.head()

In [ ]:
wgi_pivot = wgi.pivot(index=['country_name','year'], columns=['series_code'], values=['score'])

In [ ]:
wgi_pivot_df = pd.DataFrame(wgi_pivot)

In [ ]:
# wgi_pivot_df.to_csv('./data/transformed/wgi_pivot.csv')

In [ ]:
wgi_scores = pd.read_csv('./data/transformed/wgi_pivot.csv')

In [ ]:
wgi_scores.head(20) 

In [ ]:
wgi.head(20)

In [ ]:
# mm.set_index(['year', 'country'], )

In [ ]:
mm.groupby(['year'])['country'].describe()

In [ ]:
mm['stateresponse1'].value_counts()

In [ ]:
mm['protesterdemand1'].value_counts()

In [ ]:
mm['protesterdemand2'].value_counts()

In [ ]:
# mm.sort_values(by='start_date', inplace=True)

In [ ]:
mm.info()

In [ ]:
def prep_for_model(country, start_date='2006', end_date='2021'):
    df = mm[mm['country'] == country]
    df.sort_values(by='start_date', inplace=True)
    df.set_index('start_date', inplace=True)
#     df = df[(df.index >= start_date) & (mm.index <= end_date)]
    df = df.loc[start_date:end_date]
    df = pd.concat([df, pd.get_dummies(df['protesterdemand1'])], axis=1)
    df = pd.concat([df, pd.get_dummies(df['stateresponse1'])], axis=1)
    df.drop(columns = ['id', 'country', 'region', 'protestnumber', 'ccode', 'year', 'location', 'sources', 'end_date', 'participants', 'protesteridentity','protesterdemand1',
       'protesterdemand2', 'protesterdemand3', 'protesterdemand4',
       'stateresponse1', 'stateresponse2', 'stateresponse3', 'stateresponse4',
       'stateresponse5', 'stateresponse6', 'stateresponse7'], inplace=True)
    return df
#     df.to_csv(f'./data/transformed/{country}_model.csv')

In [ ]:
model_df = prep_for_model('Kyrgyzstan')

In [ ]:
fiw_kgz = fiw_clean[fiw_clean['country'] == 'Kyrgyzstan']

In [ ]:
fiw_kgz.info()

In [ ]:
pd.merge(model_df.assign(grouper=model_df.index),
               fiw_kgz.assign(grouper=fiw_kgz['fiw_edition'].dt.to_period('Y')),
               how='left', on='grouper')

In [ ]:
model_df.shape

In [ ]:
model_df.columns

In [ ]:
model_df.to_csv('./data/transformed/venezuela_model.csv')

In [ ]:
mm['stateresponse2'].value_counts()

In [ ]:
mm['participants_category'].value_counts()

In [ ]:
mm.columns

In [ ]:
mm = pd.concat([mm, pd.get_dummies(mm['protesterdemand1'])], axis=1)

In [ ]:
mm = pd.concat([mm, pd.get_dummies(mm['stateresponse1'])], axis=1)

In [ ]:
mm['protesterdemand1'].unique()

In [ ]:
mm.columns

In [ ]:
mm['police brutality'].sum()

In [ ]:
demand_cols = ['protesterdemand2', 'protesterdemand3', 'protesterdemand4']
response_cols = ['stateresponse2', 'stateresponse3', 'stateresponse4',
       'stateresponse5', 'stateresponse6', 'stateresponse7']
demand_cat = ['price increases, tax policy', 'police brutality',
       'political behavior, process', 'removal of politician',
       'labor wage dispute', 'social restrictions', 'land farm issue']

for cat in demand_cat:
    mm[cat] = [1 for category in column in category ]

In [ ]:
mm['police brutality'].sum()